### 정확도
- 정확도(Accuracy) = 예측 결과가 동일한 데이터 건수 / 전체 예측 데이터 건수
- 직관적인 평가지표
- 이진 분류의 경우 데이터 구성에 따라서 ML 모델의 성능을 왜곡할 수 있음 -정확도만으로 평가하지는 않는다
- 불균형한 레이블 값 분포에서 ML 모델의 성능을 판단할 경우는 절때 사용하지 않는다

In [2]:
### 타이타닉 데이터 셋을 이용한 분류 및 예측 - 더미클래스 생성 후 정확도 확인
import numpy as np
from sklearn.base import BaseEstimator ## 사이킷런 패키지의 모델 프레임

class MyDummyClassifier(BaseEstimator) :
    # fit 메서드는 아무것도 하지 않는다
    def fit(self, X, y=None) :
        pass
    
    ## predict()
    def predict(self, X) :
        pred = np.zeros((X.shape[0],1))
        for i in range(X.shape[0]) :
            if X['Sex'].iloc[i] == 1 :
                pred[i] = 0
            else : 
                pred[i] = 1
        return pred
    


In [3]:
## 타이타닉 전처리 함수 수행타이타닉 전처리 함수 수행
##  titanic 데이터 전처리 함수
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

# 레이블 인코딩 수행. 
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
## 원본데이터 재 로딩후 feature 데이터셋과 label 데이터 셋 추출
## 데이터
t_df = pd.read_csv('../data/titanic/titanic_train.csv')
y_t_df = t_df['Survived']
X_t_df = t_df.drop('Survived',axis=1)

X_t_df = transform_features(X_t_df)


X_train, X_test, y_train, y_test=train_test_split(X_t_df, y_t_df, \
                                                  test_size=0.2, random_state=11)


In [6]:
myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)

mypred = myclf.predict(X_test)
print('Dummy Classifier의 정확도는: {0:.4f}'.format(accuracy_score(y_test,mypred)))

Dummy Classifier의 정확도는: 0.8324


In [7]:
### Minist 데이터셋을 이진 분류 데이터셋으로 변환 후 잘못된 정확도 확인 - 레이블 분포가 편중되었을 때
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

In [18]:
class MyFakeClassifier(BaseEstimator) : 
    def fit(self,X,y) :
        pass
    
    # 입력값으로 들어오는 X 데이터셋의 크기만큼 모두 0값으로 만들어서 반환
    # 뭐가 들어오던지 전부 0으로 반환
    def predict(self,X) :
        return np.zeros((len(X),1),dtype=bool)

In [9]:
# 사이킷런 내장 데이터 셋인 load_digits() MNIST 데이터 로딩
digits = load_digits()
print(digits.data)
print('digits.data.shape : ', digits.data.shape)
print(digits.target)
print('digits.target.shape : ', digits.target.shape )

[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
digits.data.shape :  (1797, 64)
[0 1 2 ... 8 9 8]
digits.target.shape :  (1797,)


In [13]:
## 타겟변수의 값이 7인 경우 1 그나머지는 전부 0으로 타겟변수를 변환
y = (digits.target == 7).astype(int)
y[:20]
# digits.target[:20]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [14]:
X_train, X_test, y_train, y_test = train_test_split( digits.data, y, random_state=11)

In [15]:
# 불균형한 레이블 데이터 분포도 확인
print('레이블 테스트 세트 크기 : ', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

레이블 테스트 세트 크기 :  (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64


In [20]:
## Fake Classifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fpred = fakeclf.predict(X_test)
print('모든 예측을 0으로 한 정확도 : {:.3f}'.format(accuracy_score(y_test,fpred)))

모든 예측을 0으로 한 정확도 : 0.900
